In [6]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [7]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

data_directory = "r_theta/"
create_directory(data_directory)
create_directory("models/r_theta/")
create_directory("scalers")

cuda:0


In [8]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.constants import BASIC_TRAINING_COLS, FORCE_CLASS_COLS, MASS_CLASS_COLS
from isaac.training import training_loop
from isaac.models import MultiBranchModel, initialise_model
from isaac.evaluation import get_best_model_and_its_accuracy

In [9]:
BATCH_SIZE = 128
EPOCHS = 60
NORMALISE_DATA = True
STEP_SIZE = 3
SEQ_END = 2700

INPUT_DIM = 8    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

## Read dataset and preprocess it

In [10]:
all_trials = read_dataset("data/r_passive_trials.h5", cols=BASIC_TRAINING_COLS)
idx = np.random.permutation(np.arange(len(all_trials)))
train_idx = idx[:3500]
val_idx = idx[3500:4500]

train_trials = []

for index in train_idx:
    train_trials.append(all_trials[index])
    
val_trials = []

for index in val_idx:
    val_trials.append(all_trials[index])

100%|██████████| 10935/10935 [01:13<00:00, 147.97it/s]


In [6]:
RTHETA_ATTRS = [obj+"."+attr for obj in ["o1", "o2", "o3", "o4"] for attr in ["r", "theta"]]

def add_r_theta_attributes(trials):
    for trial in tqdm(trials):
        for obj in ["o1", "o2", "o3", "o4"]:
            trial[obj+".r"] = (trial[obj+".vx"]**2 + trial[obj+".vy"]**2)**0.5
            trial[obj+".theta"] = (np.arctan2(trial[obj+".vx"], trial[obj+".vy"]) * 180 / np.pi)

add_r_theta_attributes(train_trials)
add_r_theta_attributes(val_trials)

100%|██████████| 1000/1000 [01:32<00:00, 10.51it/s]


In [7]:
train_trials[0][RTHETA_ATTRS].head()

,o1.r,o1.theta,o2.r,o2.theta,o3.r,o3.theta,o4.r,o4.theta
0,8.708749,172.808526,8.649439,83.422085,5.104416,-161.452707,9.885524,151.765350
1,8.701666,172.804167,8.642233,83.422048,5.091265,-160.258772,9.961262,151.392565
2,8.694689,172.797433,8.635034,83.422084,5.082018,-159.760252,9.988861,151.251176
3,8.687808,172.788840,8.627838,83.422054,5.074320,-159.461835,10.001982,151.173335
4,8.681039,172.778130,8.620650,83.422098,5.067583,-159.266433,10.007381,151.127534


## Train model and plot loss and accuracy

In [8]:
N_MODELS = 25

stats_dfs = []
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  multiclass=True,
                                  batch_size=BATCH_SIZE, normalise_data=NORMALISE_DATA,
                                  device=device, training_columns=RTHETA_ATTRS)


for seed in range(N_MODELS):
    df = pd.DataFrame()

    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
    epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                        error,
                                                                                        loaders[0], loaders[1], 
                                                                                        EPOCHS, seq_end=SEQ_END,
                                                                                        step_size=STEP_SIZE,
                                                                                        multibranch=True)

    torch.save(best_mass_model.state_dict(), "models/r_theta/best_mass_model_seed_%d.pt" % seed)
    torch.save(best_force_model.state_dict(), "models/r_theta/best_force_model_seed_%d.pt" % seed)
    
    
    train_accuracies = np.array(epoch_accuracies[0])
    val_accuracies = np.array(epoch_accuracies[1]) 
    
    df["Epoch"] = np.arange(EPOCHS)
    df["Mass Loss"] = epoch_losses[:, 0]
    df["Force Loss"] = epoch_losses[:, 1]
    df["Mass Train Accuracy"] = train_accuracies[:, 0]
    df["Mass Val Accuracy"] = val_accuracies[:, 0]
    df["Force Train Accuracy"] = train_accuracies[:, 1]
    df["Force Val Accuracy"] = val_accuracies[:,1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"stats.h5", key="stats")

100%|██████████| 1000/1000 [00:01<00:00, 559.86it/s]
Train_loss: ([0.34667045 0.43778939]) Train_acc: ([82.25714286 74.65714286]) Val_acc: ([74.7 62. ]): 100%|██████████| 60/60 [09:13<00:00,  9.33s/it]
Train_loss: ([0.340045   0.45869921]) Train_acc: ([82.6        74.08571429]) Val_acc: ([75.6 65.9]): 100%|██████████| 60/60 [09:14<00:00,  9.29s/it]
Train_loss: ([0.34596939 0.39351491]) Train_acc: ([83.08571429 77.2       ]) Val_acc: ([73.3 63.6]): 100%|██████████| 60/60 [09:13<00:00,  9.25s/it]
Train_loss: ([0.44615042 0.51755046]) Train_acc: ([79.74285714 71.68571429]) Val_acc: ([72.8 62.9]): 100%|██████████| 60/60 [09:15<00:00,  9.28s/it]
Train_loss: ([0.36959799 0.38868694]) Train_acc: ([80.25714286 81.2       ]) Val_acc: ([70.8 64.8]): 100%|██████████| 60/60 [09:14<00:00,  9.27s/it]
Train_loss: ([0.38964769 0.48140156]) Train_acc: ([80.31428571 74.42857143]) Val_acc: ([72.1 62.7]): 100%|██████████| 60/60 [09:17<00:00,  9.17s/it]
Train_loss: ([0.77324533 0.65189486]) Train_acc: ([61

## Save model and scaler

In [9]:
joblib.dump(scaler, "scalers/passive_rtheta_scaler.sk")

['scalers/passive_rtheta_scaler.sk']